In [ ]:
#CSCE 5200 Feature Engineering Group Project (Fourier Shape Descriptor)
#Authors: Shiva Kumar Talakokula, Siddhi Vinayak

#importing all the necessary libraries.
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tkinter import filedialog, Tk, Button, Label, Scale, HORIZONTAL, StringVar
from skimage.draw import polygon2mask
from scipy.spatial import cKDTree #used for computing Chamfer Distance efficiently
from sklearn.metrics import f1_score

#Helper methods:

#Extracts largest contour from image
def extract_closed_boundary(image_path):
    # Loads original image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(" Check path or file format properly. As image can't be loaded")

    # Converts to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #Applying Canny edge detection
    edges = cv2.Canny(gray, threshold1=100, threshold2=200)

    #Morphological closing to close small gaps in edges
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    #Find contours
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not contours:
        raise ValueError("No contours detected. Try adjusting Canny thresholds.")

    #Select the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    if len(largest_contour) < 10:
        raise ValueError("Extracted contour is too small. Not a valid shape.")

    return np.squeeze(largest_contour, axis=1)


#It converts a contour into its Fourier descriptor and returns the shape centroid
def compute_fourier_descriptor(contour):
    centroid = np.mean(contour, axis=0)
    centered = contour - centroid
    contour_complex = centered[:, 0] + 1j * centered[:, 1]
    return np.fft.fft(contour_complex), centroid

#This keeps only top percentage of Fourier coefficients by magnitude
def filter_top_coefficients(descriptors, percent):
    descriptors_copy = np.copy(descriptors)
    n = len(descriptors)
    keep = max(2, int(n * percent / 100))  #At least 2 to avoid total collapse

    #Get indices of top 'keep' coefficients by magnitude
    top_indices = np.argsort(np.abs(descriptors_copy))[::-1][:keep]

    #It creates a zero array and restore only top coefficients
    filtered = np.zeros_like(descriptors_copy)
    filtered[top_indices] = descriptors_copy[top_indices]
    return filtered

#Rebuilds the shape from its truncated Fourier descriptor
def reconstruct_contour(truncated_fd, centroid):
    inverse = np.fft.ifft(truncated_fd)
    contour_reconstructed = np.stack((inverse.real, inverse.imag), axis=-1)
    contour_reconstructed += centroid
    return np.int32(contour_reconstructed)

#Resamples the contour to a fixed number of points for consistency
def smooth_and_resample_contour(contour, num_points=500):
    if len(contour.shape) != 2 or contour.shape[1] != 2:
        raise ValueError("Contour must be Nx2 array.")
    contour = contour.astype(np.float32)
    distances = np.cumsum(np.sqrt(np.sum(np.diff(contour, axis=0)**2, axis=1)))
    distances = np.insert(distances, 0, 0)
    uniform_distances = np.linspace(0, distances[-1], num_points)
    resampled = np.empty((num_points, 2))
    resampled[:, 0] = np.interp(uniform_distances, distances, contour[:, 0])
    resampled[:, 1] = np.interp(uniform_distances, distances, contour[:, 1])
    return resampled

# This is used to save Fourier descriptor as .npy file
def save_descriptor(fd, filename="fourier_descriptor.npy"):
    np.save(filename, fd)

#Computes Chamfer distance between two sets of points using KD-trees
def chamfer_distance(a, b):
    a_tree = cKDTree(a)
    b_tree = cKDTree(b)
    d_ab, _ = a_tree.query(b)
    d_ba, _ = b_tree.query(a)
    return np.mean(d_ab) + np.mean(d_ba)

# Calculates Intersection-over-Union between masks of original and reconstructed shapes
def compute_iou(original, reconstructed, shape):
    mask1 = polygon2mask(shape, original)
    mask2 = polygon2mask(shape, reconstructed)
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return intersection / union if union != 0 else 0

# Computes F1 score comparing shape boundaries as masks
def compute_boundary_f1(original, reconstructed, shape):
    mask1 = polygon2mask(shape, original)
    mask2 = polygon2mask(shape, reconstructed)
    return f1_score(mask1.flatten(), mask2.flatten())

# Simple GUI app to load an image, extract shape contour, apply Fourier descriptors, and visualize reconstruction purposes.
class FourierGUI:
    def __init__(self):
        self.root = Tk()
        self.root.title("Fourier Shape Descriptor GUI")

        self.label = Label(self.root, text="Load an image to begin")
        self.label.pack()

        self.load_button = Button(self.root, text="Load Image", command=self.load_image)
        self.load_button.pack()

        self.slider_label = Label(self.root, text="Percentage of Fourier Coefficients:")
        self.slider_label.pack()

        self.slider = Scale(self.root, from_=1, to=100, orient=HORIZONTAL)
        self.slider.set(50)
        self.slider.pack()

        self.reconstruct_button = Button(self.root, text="Reconstruct", command=self.reconstruct_image)
        self.reconstruct_button.pack()

        self.save_button = Button(self.root, text="Save Descriptor", command=self.save_fd)
        self.save_button.pack()

        self.image_path = None
        self.contour = None
        self.fd = None
        self.contour_centroid = None

        self.root.mainloop()

    # Loads image and extracts the main shape contour
    def load_image(self):
        self.image_path = filedialog.askopenfilename()
        if not self.image_path:
            return
        raw_contour = extract_closed_boundary(self.image_path)
        resampled = smooth_and_resample_contour(raw_contour)
        self.contour = resampled
        self.fd, self.contour_centroid = compute_fourier_descriptor(resampled)
        plt.figure()
        plt.title("Original Contour")
        plt.plot(self.contour[:, 0], self.contour[:, 1])
        plt.gca().invert_yaxis()
        plt.axis("equal")
        plt.show()

    # Reconstructs contour using selected percentage of Fourier coefficients and shows comparison metrics
    def reconstruct_image(self):
        if self.fd is None or self.contour is None:
            return
        percent = self.slider.get()
        # Truncate Fourier coefficients by magnitude
        fd_truncated = filter_top_coefficients(self.fd, percent)
        # Reconstruct contour using inverse FFT + add centroid back
        reconstructed = reconstruct_contour(fd_truncated, self.contour_centroid)

        # Show plot
        plt.figure()
        plt.title(f"Reconstructed with {percent}% coefficients")
        plt.plot(reconstructed[:, 0], reconstructed[:, 1])
        plt.gca().invert_yaxis()
        plt.axis("equal")
        plt.show()

        #Load original image shape
        img = cv2.imread(self.image_path, cv2.IMREAD_GRAYSCALE)

        # Compute metrics
        iou = compute_iou(self.contour, reconstructed, shape=img.shape)
        chamfer = chamfer_distance(self.contour, reconstructed)
        f1 = compute_boundary_f1(self.contour, reconstructed, shape=img.shape)

        #Display metrics
        print(f"[Metrics for {percent}% coefficients]")
        print(f"Chamfer Distance: {chamfer:.2f}")
        print(f"IoU Score: {iou:.3f}")
        print(f"Boundary F1 Score: {f1:.3f}")


    # Saves current Fourier descriptor to file
    def save_fd(self):
        if self.fd is not None:
            save_descriptor(self.fd)
            print("Fourier descriptor saved as 'fourier_descriptor.npy'")

if __name__ == "__main__":
    FourierGUI()
